In [1]:
# Install required packages
!pip install ultralytics opencv-python yt-dlp matplotlib numpy --quiet

In [2]:
import yt_dlp

# Change this URL to any sports video you like
url = "https://www.youtube.com/shorts/pCTxUWW42uU"

ydl_opts = {
    'format': 'mp4',
    'outtmpl': 'sample.mp4',
    'quiet': True
}

with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    ydl.download([url])

print('✅ Video downloaded: sample.mp4')

✅ Video downloaded: sample.mp4


In [3]:
# ─────────────────────────────────────────────
#   CONFIGURATION — Edit these values
# ─────────────────────────────────────────────

VIDEO_PATH   = "sample.mp4"   # Path to your input video
OUTPUT_PATH  = "output_risk.mp4"  # Where to save the annotated video
USE_WEBCAM   = False          # Set True for real-time webcam mode (Google Colab doesn't support this)
MAX_FRAMES   = 500            # Process up to this many frames (None = all frames)
MODEL_SIZE   = "n"            # YOLOv8 size: n=nano (fast), s=small, m=medium
CONF_THRESH  = 0.5            # Keypoint confidence threshold
SHOW_ANGLES  = True           # Draw angle values on joints
SHOW_GRAPH   = True           # Overlay a mini risk history graph
# ─────────────────────────────────────────────

In [4]:
import cv2
import numpy as np
import math
import matplotlib
matplotlib.use('Agg')  # Non-interactive backend for Colab
import matplotlib.pyplot as plt
from ultralytics import YOLO
from collections import deque
import warnings
warnings.filterwarnings('ignore')

# Load YOLOv8 Pose model
model = YOLO(f"yolov8{MODEL_SIZE}-pose.pt")
print(f"✅ YOLOv8{MODEL_SIZE.upper()}-Pose model loaded")

✅ YOLOv8N-Pose model loaded


In [5]:
# ─────────────────────────────────────────────────────────────────────
#  YOLOv8 Pose Keypoint Index Reference
#  0=nose  1=l_eye  2=r_eye  3=l_ear  4=r_ear
#  5=l_shoulder  6=r_shoulder  7=l_elbow  8=r_elbow
#  9=l_wrist  10=r_wrist  11=l_hip  12=r_hip
#  13=l_knee  14=r_knee  15=l_ankle  16=r_ankle
# ─────────────────────────────────────────────────────────────────────

def calculate_angle(a, b, c):
    """Angle at joint B, formed by A-B-C (in degrees)."""
    a, b, c = np.array(a, float), np.array(b, float), np.array(c, float)
    ba, bc = a - b, c - b
    norm_ba, norm_bc = np.linalg.norm(ba), np.linalg.norm(bc)
    if norm_ba == 0 or norm_bc == 0:
        return 180.0
    cosine = np.dot(ba, bc) / (norm_ba * norm_bc)
    return float(np.degrees(np.arccos(np.clip(cosine, -1.0, 1.0))))


def trunk_lean_angle(shoulder_mid, hip_mid):
    """Angle of the torso line from vertical (0° = perfectly upright)."""
    dx = hip_mid[0] - shoulder_mid[0]
    dy = hip_mid[1] - shoulder_mid[1]
    if dy == 0:
        return 0.0
    angle_from_vertical = abs(math.degrees(math.atan2(dx, dy)))
    return angle_from_vertical


def is_visible(kp, conf=None, threshold=CONF_THRESH):
    """Check if a keypoint is reliably detected (non-zero coords)."""
    return kp[0] > 0 and kp[1] > 0


def analyze_person(person_kp, person_conf):
    """
    Given keypoints (Nx2) and confidences (N,) for one person,
    return a dict of risk scores (0-100) per factor, plus composite.
    """
    risks = {}
    details = {}  # For displaying angles on frame

    # ── Helper: get keypoint if confident ──
    def kp(idx):
        return person_kp[idx] if (person_conf[idx] > CONF_THRESH
                                   and is_visible(person_kp[idx])) else None

    l_shoulder = kp(5);  r_shoulder = kp(6)
    l_hip      = kp(11); r_hip      = kp(12)
    l_knee     = kp(13); r_knee     = kp(14)
    l_ankle    = kp(15); r_ankle    = kp(16)
    l_elbow    = kp(7);  r_elbow    = kp(8)

    # ── 1. LEFT KNEE ANGLE (ACL / knee valgus risk) ──
    if all(v is not None for v in [l_hip, l_knee, l_ankle]):
        angle = calculate_angle(l_hip, l_knee, l_ankle)
        details['L_Knee'] = (l_knee, angle)
        if angle < 120:
            risks['L_ACL'] = 90
        elif angle < 140:
            risks['L_ACL'] = 60
        elif angle < 160:
            risks['L_ACL'] = 30
        else:
            risks['L_ACL'] = 0

    # ── 2. RIGHT KNEE ANGLE ──
    if all(v is not None for v in [r_hip, r_knee, r_ankle]):
        angle = calculate_angle(r_hip, r_knee, r_ankle)
        details['R_Knee'] = (r_knee, angle)
        if angle < 120:
            risks['R_ACL'] = 90
        elif angle < 140:
            risks['R_ACL'] = 60
        elif angle < 160:
            risks['R_ACL'] = 30
        else:
            risks['R_ACL'] = 0

    # ── 3. LEFT HIP FLEXION ──
    if all(v is not None for v in [l_shoulder, l_hip, l_knee]):
        angle = calculate_angle(l_shoulder, l_hip, l_knee)
        details['L_Hip'] = (l_hip, angle)
        if angle < 100:
            risks['L_Hip'] = 80
        elif angle < 130:
            risks['L_Hip'] = 45
        elif angle < 150:
            risks['L_Hip'] = 20
        else:
            risks['L_Hip'] = 0

    # ── 4. RIGHT HIP FLEXION ──
    if all(v is not None for v in [r_shoulder, r_hip, r_knee]):
        angle = calculate_angle(r_shoulder, r_hip, r_knee)
        details['R_Hip'] = (r_hip, angle)
        if angle < 100:
            risks['R_Hip'] = 80
        elif angle < 130:
            risks['R_Hip'] = 45
        elif angle < 150:
            risks['R_Hip'] = 20
        else:
            risks['R_Hip'] = 0

    # ── 5. TRUNK / SPINE LEAN ──
    if all(v is not None for v in [l_shoulder, r_shoulder, l_hip, r_hip]):
        shoulder_mid = ((l_shoulder[0] + r_shoulder[0]) / 2,
                        (l_shoulder[1] + r_shoulder[1]) / 2)
        hip_mid = ((l_hip[0] + r_hip[0]) / 2,
                   (l_hip[1] + r_hip[1]) / 2)
        lean = trunk_lean_angle(shoulder_mid, hip_mid)
        details['Trunk'] = (tuple(map(int, shoulder_mid)), lean)
        if lean > 35:
            risks['Trunk'] = 75
        elif lean > 25:
            risks['Trunk'] = 50
        elif lean > 15:
            risks['Trunk'] = 25
        else:
            risks['Trunk'] = 0

    # ── 6. SHOULDER ASYMMETRY ──
    if all(v is not None for v in [l_shoulder, r_shoulder]):
        diff = abs(l_shoulder[1] - r_shoulder[1])
        if diff > 40:
            risks['Shoulder_Asym'] = 70
        elif diff > 25:
            risks['Shoulder_Asym'] = 40
        elif diff > 15:
            risks['Shoulder_Asym'] = 20
        else:
            risks['Shoulder_Asym'] = 0

    # ── 7. HIP ASYMMETRY ──
    if all(v is not None for v in [l_hip, r_hip]):
        diff = abs(l_hip[1] - r_hip[1])
        if diff > 40:
            risks['Hip_Asym'] = 70
        elif diff > 25:
            risks['Hip_Asym'] = 40
        elif diff > 15:
            risks['Hip_Asym'] = 20
        else:
            risks['Hip_Asym'] = 0

    # ── 8. ANKLE DORSIFLEXION (L) ──
    if all(v is not None for v in [l_knee, l_ankle]):
        # Virtual toe point: extend ankle direction downward
        virtual_toe = (l_ankle[0] + (l_ankle[0] - l_knee[0]) * 0.3,
                       l_ankle[1] + 30)
        angle = calculate_angle(l_knee, l_ankle, virtual_toe)
        if angle > 120:
            risks['L_Ankle'] = 60
        elif angle > 110:
            risks['L_Ankle'] = 30
        else:
            risks['L_Ankle'] = 0

    # ── 9. ANKLE DORSIFLEXION (R) ──
    if all(v is not None for v in [r_knee, r_ankle]):
        virtual_toe = (r_ankle[0] + (r_ankle[0] - r_knee[0]) * 0.3,
                       r_ankle[1] + 30)
        angle = calculate_angle(r_knee, r_ankle, virtual_toe)
        if angle > 120:
            risks['R_Ankle'] = 60
        elif angle > 110:
            risks['R_Ankle'] = 30
        else:
            risks['R_Ankle'] = 0

    # ── COMPOSITE RISK SCORE (weighted average) ──
    weights = {
        'L_ACL': 0.25, 'R_ACL': 0.25,
        'L_Hip': 0.10, 'R_Hip': 0.10,
        'Trunk': 0.12,
        'Shoulder_Asym': 0.05, 'Hip_Asym': 0.05,
        'L_Ankle': 0.04, 'R_Ankle': 0.04
    }

    total_weight = sum(weights[k] for k in risks if k in weights)
    if total_weight > 0:
        composite = sum(risks[k] * weights[k] for k in risks if k in weights) / total_weight
    else:
        composite = 0

    return risks, min(int(composite), 100), details


print("✅ Biomechanical analysis engine ready")

✅ Biomechanical analysis engine ready


In [6]:
def risk_color(score):
    """Return BGR color: green → yellow → red based on score 0-100."""
    if score < 30:
        return (0, 200, 0)       # Green
    elif score < 60:
        return (0, 200, 255)     # Yellow-Orange
    elif score < 80:
        return (0, 100, 255)     # Orange
    else:
        return (0, 0, 230)       # Red


def risk_label(score):
    if score < 30:  return "LOW"
    if score < 60:  return "MODERATE"
    if score < 80:  return "HIGH"
    return "CRITICAL"


def draw_dashboard(frame, composite, risks, person_idx=0, frame_num=0):
    """Draw a semi-transparent HUD dashboard on the frame."""
    h, w = frame.shape[:2]
    overlay = frame.copy()

    # Background panel
    panel_w, panel_h = 260, 230
    cv2.rectangle(overlay, (10, 10), (10 + panel_w, 10 + panel_h), (20, 20, 20), -1)
    cv2.addWeighted(overlay, 0.6, frame, 0.4, 0, frame)

    # Title
    cv2.putText(frame, f"INJURY RISK — P{person_idx+1}",
                (18, 35), cv2.FONT_HERSHEY_DUPLEX, 0.55, (200, 200, 200), 1)

    # Composite score bar
    bar_x, bar_y, bar_w, bar_h = 18, 45, 230, 18
    cv2.rectangle(frame, (bar_x, bar_y), (bar_x + bar_w, bar_y + bar_h), (60, 60, 60), -1)
    fill = int(bar_w * composite / 100)
    cv2.rectangle(frame, (bar_x, bar_y), (bar_x + fill, bar_y + bar_h), risk_color(composite), -1)
    cv2.rectangle(frame, (bar_x, bar_y), (bar_x + bar_w, bar_y + bar_h), (120, 120, 120), 1)
    label = f"{risk_label(composite)}  {composite}%"
    cv2.putText(frame, label, (bar_x + 4, bar_y + 14),
                cv2.FONT_HERSHEY_SIMPLEX, 0.48, (255, 255, 255), 1)

    # Per-factor bars
    factor_labels = {
        'L_ACL': 'L Knee (ACL)', 'R_ACL': 'R Knee (ACL)',
        'L_Hip': 'L Hip Flex',   'R_Hip': 'R Hip Flex',
        'Trunk': 'Trunk Lean',
        'Shoulder_Asym': 'Shldr Asym', 'Hip_Asym': 'Hip Asym'
    }
    y_offset = 78
    for key, disp in factor_labels.items():
        val = risks.get(key, None)
        if val is None:
            continue
        mini_bar_w = 100
        fill_w = int(mini_bar_w * val / 100)
        cv2.putText(frame, disp + ":", (18, y_offset),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.38, (180, 180, 180), 1)
        bx = 130
        cv2.rectangle(frame, (bx, y_offset - 9), (bx + mini_bar_w, y_offset + 2), (60, 60, 60), -1)
        cv2.rectangle(frame, (bx, y_offset - 9), (bx + fill_w, y_offset + 2), risk_color(val), -1)
        cv2.putText(frame, str(val), (bx + mini_bar_w + 4, y_offset),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.38, (200, 200, 200), 1)
        y_offset += 18

    # Frame counter
    cv2.putText(frame, f"Frame: {frame_num}",
                (w - 120, h - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (160, 160, 160), 1)

    return frame


def draw_angle_on_joint(frame, joint_pos, angle_val, color):
    """Display angle value next to a joint."""
    if joint_pos is None:
        return
    x, y = int(joint_pos[0]), int(joint_pos[1])
    cv2.putText(frame, f"{angle_val:.0f}°",
                (x + 8, y - 8), cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 1, cv2.LINE_AA)


def draw_risk_graph(frame, history, max_frames=80):
    """Draw a mini risk history sparkline in the bottom-right corner."""
    h, w = frame.shape[:2]
    gw, gh = 180, 60
    gx, gy = w - gw - 10, h - gh - 25

    # Semi-transparent background
    overlay = frame.copy()
    cv2.rectangle(overlay, (gx, gy), (gx + gw, gy + gh), (20, 20, 20), -1)
    cv2.addWeighted(overlay, 0.55, frame, 0.45, 0, frame)

    cv2.putText(frame, "Risk History", (gx + 4, gy - 4),
                cv2.FONT_HERSHEY_SIMPLEX, 0.38, (160, 160, 160), 1)

    # Draw threshold lines
    for pct, col in [(0.3, (0, 200, 0)), (0.6, (0, 200, 255)), (0.8, (0, 0, 230))]:
        ly = int(gy + gh - pct * gh)
        cv2.line(frame, (gx, ly), (gx + gw, ly), col, 1)

    # Plot line
    recent = list(history)[-max_frames:]
    if len(recent) > 1:
        for i in range(1, len(recent)):
            x1 = int(gx + (i - 1) / max_frames * gw)
            x2 = int(gx + i / max_frames * gw)
            y1 = int(gy + gh - recent[i - 1] / 100 * gh)
            y2 = int(gy + gh - recent[i] / 100 * gh)
            cv2.line(frame, (x1, y1), (x2, y2), (255, 255, 255), 1, cv2.LINE_AA)

    return frame


print("✅ Visualization utilities ready")

✅ Visualization utilities ready


In [7]:
# Open video source
source = 0 if USE_WEBCAM else VIDEO_PATH
cap = cv2.VideoCapture(source)

if not cap.isOpened():
    raise FileNotFoundError(f"❌ Could not open video: {source}")

width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps    = cap.get(cv2.CAP_PROP_FPS) or 30

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out    = cv2.VideoWriter(OUTPUT_PATH, fourcc, fps, (width, height))

risk_history    = deque(maxlen=200)  # Rolling risk history
frame_count     = 0
all_risk_scores = []   # For final summary

print(f"📹 Processing: {source}  [{width}x{height} @ {fps:.0f}fps]")
print(f"   Output  → {OUTPUT_PATH}")
print(f"   Max frames: {MAX_FRAMES or 'all'}")
print()

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1
    if MAX_FRAMES and frame_count > MAX_FRAMES:
        break

    # ── Run YOLOv8 Pose inference ──
    results = model(frame, verbose=False)

    annotated = frame.copy()
    frame_max_risk = 0

    for r in results:
        if r.keypoints is None or r.keypoints.xy is None:
            continue

        # Draw default YOLO skeleton
        annotated = r.plot(img=annotated, kpt_radius=3, line_width=1)

        keypoints = r.keypoints.xy.cpu().numpy()    # shape: (N_persons, 17, 2)
        confs     = r.keypoints.conf.cpu().numpy()  # shape: (N_persons, 17)

        for person_idx, (person_kp, person_conf) in enumerate(zip(keypoints, confs)):
            risks, composite, details = analyze_person(person_kp, person_conf)

            frame_max_risk = max(frame_max_risk, composite)

            # Draw angle labels near joints
            if SHOW_ANGLES:
                for joint_name, (jpos, angle_val) in details.items():
                    r_key = {'L_Knee': 'L_ACL', 'R_Knee': 'R_ACL',
                             'L_Hip': 'L_Hip', 'R_Hip': 'R_Hip',
                             'Trunk': 'Trunk'}.get(joint_name, joint_name)
                    jcolor = risk_color(risks.get(r_key, 0))
                    draw_angle_on_joint(annotated, jpos, angle_val, jcolor)

            # Draw per-person HUD dashboard
            annotated = draw_dashboard(annotated, composite, risks, person_idx, frame_count)

    # Update history & draw sparkline
    risk_history.append(frame_max_risk)
    all_risk_scores.append(frame_max_risk)

    if SHOW_GRAPH:
        annotated = draw_risk_graph(annotated, risk_history)

    out.write(annotated)

    # Progress update every 50 frames
    if frame_count % 50 == 0:
        avg_so_far = np.mean(all_risk_scores) if all_risk_scores else 0
        print(f"  Frame {frame_count:4d}  |  Avg Risk: {avg_so_far:.1f}%  |  Current: {frame_max_risk}%")

cap.release()
out.release()

# ── Verify the output file was written ──
import os, json
if os.path.exists(OUTPUT_PATH) and os.path.getsize(OUTPUT_PATH) > 0:
    print(f"\n✅ Done! Processed {frame_count} frames → {OUTPUT_PATH}")
    print(f"   File size: {os.path.getsize(OUTPUT_PATH)/1024:.1f} KB")
else:
    print("\n❌ Output file missing or empty — check for errors above.")

# Save risk scores to disk so Cell 9 can load them even if kernel resets
with open('risk_scores.json', 'w') as f:
    json.dump(all_risk_scores, f)
print(f"   Risk scores saved → risk_scores.json ({len(all_risk_scores)} entries)")

📹 Processing: sample.mp4  [360x640 @ 30fps]
   Output  → output_risk.mp4
   Max frames: 500

  Frame   50  |  Avg Risk: 74.0%  |  Current: 74%
  Frame  100  |  Avg Risk: 72.1%  |  Current: 60%
  Frame  150  |  Avg Risk: 63.6%  |  Current: 49%
  Frame  200  |  Avg Risk: 57.9%  |  Current: 36%
  Frame  250  |  Avg Risk: 52.0%  |  Current: 19%
  Frame  300  |  Avg Risk: 45.6%  |  Current: 7%
  Frame  350  |  Avg Risk: 42.9%  |  Current: 49%
  Frame  400  |  Avg Risk: 44.8%  |  Current: 62%
  Frame  450  |  Avg Risk: 47.2%  |  Current: 66%
  Frame  500  |  Avg Risk: 49.2%  |  Current: 69%

✅ Done! Processed 501 frames → output_risk.mp4
   File size: 10246.9 KB
   Risk scores saved → risk_scores.json (500 entries)


In [8]:
# Re-encode with H.264 so Colab can play it inline
import os

# Verify source file exists before running ffmpeg
if not os.path.exists(OUTPUT_PATH):
    raise FileNotFoundError(f"❌ Source file '{OUTPUT_PATH}' not found. Did Cell 7 complete successfully?")

print(f"📦 Source file size: {os.path.getsize(OUTPUT_PATH)/1024:.1f} KB")

# Use shell variable expansion safely (avoids brace-expansion issues in some shells)
src = OUTPUT_PATH
!ffmpeg -y -i "$src" -vcodec libx264 -crf 23 -preset fast -acodec aac output_h264.mp4 -loglevel error

if os.path.exists('output_h264.mp4') and os.path.getsize('output_h264.mp4') > 0:
    print(f"✅ Re-encoded → output_h264.mp4  ({os.path.getsize('output_h264.mp4')/1024:.1f} KB)")
else:
    print("❌ Re-encoding failed. Try running: !ffmpeg -y -i output_risk.mp4 -vcodec libx264 output_h264.mp4")

📦 Source file size: 10246.9 KB
✅ Re-encoded → output_h264.mp4  (2127.9 KB)


In [9]:
import matplotlib.pyplot as plt
import numpy as np
import os, json

# ── Load risk scores: from memory (if Cell 7 ran) or from saved file ──
if 'all_risk_scores' not in dir() or len(all_risk_scores) == 0:
    if os.path.exists('risk_scores.json'):
        with open('risk_scores.json') as f:
            all_risk_scores = json.load(f)
        print(f"📂 Loaded {len(all_risk_scores)} scores from risk_scores.json")
    else:
        raise RuntimeError("❌ No risk scores found. Please run Cell 7 (main processing loop) first.")

scores = np.array(all_risk_scores)
if len(scores) == 0:
    raise ValueError("❌ Risk scores list is empty. The video may have had no detections.")

print("═" * 45)
print("  📊  INJURY RISK ANALYSIS SUMMARY")
print("═" * 45)
print(f"  Total frames analyzed : {len(scores)}")
print(f"  Average risk score    : {scores.mean():.1f}%")
print(f"  Peak risk score       : {scores.max():.0f}%")
print(f"  Frames LOW  (<30%)    : {(scores < 30).sum()} ({100*(scores<30).mean():.0f}%)")
print(f"  Frames MOD  (30-60%)  : {((scores>=30)&(scores<60)).sum()} ({100*((scores>=30)&(scores<60)).mean():.0f}%)")
print(f"  Frames HIGH (60-80%)  : {((scores>=60)&(scores<80)).sum()} ({100*((scores>=60)&(scores<80)).mean():.0f}%)")
print(f"  Frames CRIT (>80%)    : {(scores>=80).sum()} ({100*(scores>=80).mean():.0f}%)")
print("═" * 45)

# Plot risk over time
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 4))
fig.patch.set_facecolor('#1a1a2e')

# Risk over time
ax1.set_facecolor('#16213e')
ax1.plot(scores, color='#e94560', linewidth=0.8, alpha=0.9, label='Risk Score')
ax1.fill_between(range(len(scores)), scores, alpha=0.15, color='#e94560')
ax1.axhline(30, color='#00c853', linestyle='--', linewidth=1, alpha=0.7, label='Low/Mod threshold')
ax1.axhline(60, color='#ffd600', linestyle='--', linewidth=1, alpha=0.7, label='Mod/High threshold')
ax1.axhline(80, color='#dd2c00', linestyle='--', linewidth=1, alpha=0.7, label='High/Critical threshold')
ax1.set_xlim(0, len(scores))
ax1.set_ylim(0, 100)
ax1.set_xlabel('Frame', color='white')
ax1.set_ylabel('Composite Risk Score (%)', color='white')
ax1.set_title('Injury Risk Over Time', color='white', fontsize=13)
ax1.tick_params(colors='white')
ax1.legend(fontsize=8, facecolor='#1a1a2e', labelcolor='white')
for sp in ax1.spines.values(): sp.set_color('#444')

# Risk distribution
ax2.set_facecolor('#16213e')
categories = ['LOW\n(<30)', 'MODERATE\n(30-60)', 'HIGH\n(60-80)', 'CRITICAL\n(>80)']
counts = [
    (scores < 30).sum(),
    ((scores >= 30) & (scores < 60)).sum(),
    ((scores >= 60) & (scores < 80)).sum(),
    (scores >= 80).sum()
]
colors_bar = ['#00c853', '#ffd600', '#ff6d00', '#dd2c00']
bars = ax2.bar(categories, counts, color=colors_bar, edgecolor='#444', linewidth=0.8)
for bar, cnt in zip(bars, counts):
    if cnt > 0:
        ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
                 str(cnt), ha='center', va='bottom', color='white', fontsize=10)
ax2.set_ylabel('Frame Count', color='white')
ax2.set_title('Risk Level Distribution', color='white', fontsize=13)
ax2.tick_params(colors='white')
for sp in ax2.spines.values(): sp.set_color('#444')

plt.tight_layout()
plt.savefig('risk_summary.png', dpi=120, bbox_inches='tight', facecolor='#1a1a2e')
plt.show()
print("\n✅ Summary chart saved: risk_summary.png")

═════════════════════════════════════════════
  📊  INJURY RISK ANALYSIS SUMMARY
═════════════════════════════════════════════
  Total frames analyzed : 500
  Average risk score    : 49.2%
  Peak risk score       : 79%
  Frames LOW  (<30%)    : 116 (23%)
  Frames MOD  (30-60%)  : 173 (35%)
  Frames HIGH (60-80%)  : 211 (42%)
  Frames CRIT (>80%)    : 0 (0%)
═════════════════════════════════════════════

✅ Summary chart saved: risk_summary.png


In [10]:
from IPython.display import Video, display

# Play inline
display(Video("output_h264.mp4", embed=True, width=400))

In [11]:
# Download the annotated video and summary chart
from google.colab import files

files.download("output_h264.mp4")
files.download("risk_summary.png")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>